## Correlate exposures with telemetry

See the introductory notes in the [Accessing_EFD_data.ipynb](https://github.com/lsst-sqre/notebook-demo/blob/master/efd_examlpes/Accessing_EFD_data.ipynb) notebook for links to documentation and help with authentication.

In [ ]:
import matplotlib
%matplotlib inline
from matplotlib import pylab as plt
import pandas as pd
from astropy.time import Time, TimeDelta

from lsst_efd_client import EfdClient

In [ ]:
client = EfdClient('ldf_stable_efd')

Specify a time range for data in `InfluxQL`.  This notebook looks at data from October 2021.

In [ ]:
t2 = Time('2021-10-06T18:00:00')
t1 = t2 - TimeDelta(24*3600, format='sec')

Query each of the measurements we may want to correlate later in the notebook.  Note that this could be done as a single query, but the result is a dictionary of `DataFrames` which I find less convenient to use than named variables corresponding to one `DataFrame` each.

In [ ]:
df1 = await client.select_time_series("lsst.sal.ATCamera.fp_Reb", ["temp1", "temp2", "temp3", "temp4", "temp5", "temp6", "aspicl_Temp0"], t1, t2)

df2 = await client.select_time_series("lsst.sal.ATCamera.command_takeImages", ["expTime", "numImages"], t1, t2)

df3 = await client.select_time_series("lsst.sal.ATCamera.fp_Reb", "anaI", t1, t2)

shutter_close = await client.select_time_series("lsst.sal.ATCamera.logevent_endShutterClose", "priority", t1, t2)

shutter_open = await client.select_time_series("lsst.sal.ATCamera.logevent_endShutterOpen", "priority", t1, t2)

start_readout = await client.select_time_series("lsst.sal.ATCamera.logevent_startReadout", ["priority", "private_sndStamp", "private_rcvStamp"], t1, t2)

end_readout = await client.select_time_series("lsst.sal.ATCamera.logevent_endReadout", ["priority", "private_sndStamp", "private_rcvStamp"], t1, t2)

start_integration = await client.select_time_series("lsst.sal.ATCamera.logevent_startIntegration", ["priority", "private_sndStamp", "private_rcvStamp"], t1, t2)

Select a telemetry stream, `anaI`, and overplot log messages from the `start_integration`, `start_readout`, and `end_readout` log streams.

In [ ]:
plt = df3.plot(y="anaI")
for i, r in start_integration.iterrows():
    plt.axvline(pd.Timestamp(r['private_sndStamp'], unit='s'), c='g', alpha=0.3)
for i, r in start_readout.iterrows():
    plt.axvline(pd.Timestamp(r['private_sndStamp'], unit='s'), c='b', alpha=0.3)
for i, r in end_readout.iterrows():
    plt.axvline(pd.Timestamp(r['private_sndStamp'], unit='s'), c='r', alpha=0.3)

Inspect time stamps.  There are three obvious ones: the timestamp from `InfluxDB`, `private_rcvStamp` from SAL, and `private_sndStamp` from SAL.

In [ ]:
plt = df3.plot()
diff = []
for i, r in start_integration.iterrows():
    plt.axvline(i, c='g', alpha=0.3)
    plt.axvline(pd.Timestamp(r['private_rcvStamp'], unit='s'), c='b', alpha=0.3)
    plt.axvline(pd.Timestamp(r['private_sndStamp'], unit='s'), c='r', alpha=0.3)
    diff.append(pd.Timestamp(r['private_rcvStamp'], unit='s') - pd.Timestamp(r['private_sndStamp'], unit='s'))
    diff[-1] = diff[-1].value/1000000000

Look at the histogram of timestamp differences for `private_rcvStamp` - `private_sndStamp`.

In [ ]:
diff_df = pd.DataFrame(diff)
diff_df.hist(bins=50)